# ML Frameworks Interoperability Cheat Sheet
## Introduction

This notebook is an appendix to the <a href="https://developer.nvidia.com/blog/machine-learning-frameworks-interoperability-part-1-memory-layouts-and-memory-pools/">Machine Learning Frameworks Interoperability</a> blog series. It aims to be a lookup table when converting data between the following ML frameworks: <a href="https://pandas.pydata.org/">pandas</a>, <a href="https://numpy.org">NumPy</a>, <a href="https://github.com/rapidsai/cudf">cuDF</a>, <a href="https://cupy.dev/">CuPy</a>, <a href="https://github.com/google/jax">JAX</a>, <a href="https://numba.pydata.org/">Numba</a>, <a href="https://www.tensorflow.org">TensorFlow</a>, <a href="https://pytorch.org/">PyTorch</a> and <a href="https://mxnet.apache.org/">MXNet</a>.

In order to make it easier to have all those libraries up and running, we have used the RAPIDS 22.10 container on Ubuntu 20.04 as a base container, and then added a few missing libraries via <a href="https://pip.pypa.io/en/stable/cli/pip_install/">pip install</a>. We encourage you to run this notebook on the latest RAPIDS container. Alternatively, you can also set up a <a href="https://docs.conda.io/projects/conda/en/latest/user-guide/index.html">conda virtual environment</a>. In both cases, please visit <a href="https://rapids.ai/start.html#get-rapids">RAPIDS release selector</a> for installation details.

Finally, please find below the details of the container we used when creating this notebook. For reproducibility purposes, please use the following command:

```console
foo@bar:~$ docker pull nvcr.io/nvidia/rapidsai/rapidsai:22.10-cuda11.4-runtime-ubuntu18.04-py3.9
foo@bar:~$ docker run --gpus all --rm -it --shm-size=1g --ulimit memlock=-1 -p 8888:8888 -p 8787:8787 -p 8786:8786 \
           nvcr.io/nvidia/rapidsai/rapidsai:22.10-cuda11.4-runtime-ubuntu18.04-py3.9
```

# Install missing dependencies

In [1]:
# Jax install
print("Installing Jax. It usually takes less than one minute. Please, bear with us.")
!pip -q install --root-user-action=ignore --upgrade jax==0.3.23 jaxlib==0.3.22+cuda11_cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

# PyTorch install
print("Installing PyTorch. It usually takes less than two minutes. Please, be patient.")
!pip -q install --root-user-action=ignore --upgrade torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

# MXNet + cuDNN install
print("Installing MXNet + cuDNN. It usually takes less than one minute. Please, hang on.")
!pip -q install --root-user-action=ignore --upgrade mxnet-cu112
!wget -q -nc https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcudnn8_8.6.0.163-1+cuda11.8_amd64.deb
!dpkg -i libcudnn8_8.6.0.163-1+cuda11.8_amd64.deb > /dev/null

# TensorFlow install
print("Installing TensorFlow. It usually takes less than one minute. Please, mind the gap.")
print("Oops, sorry, wrong translation. Please, hold on. :-)")
!pip -q install --root-user-action=ignore --upgrade tensorflow==2.8.0

print("Done! Great job!")

Installing Jax. It usually takes less than one minute. Please, bear with us.
Installing PyTorch. It usually takes less than two minutes. Please, be patient.
Installing MXNet + cuDNN. It usually takes less than one minute. Please, hang on.
Installing TensorFlow. It usually takes less than one minute. Please, mind the gap.
Oops, sorry, wrong translation. Please, hold on. :-)
Done! Great job!


In [2]:
import cudf
import cupy as cp
import jax
import jax.dlpack
import jax.numpy as jnp
import mxnet as mx
import numba as nb
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import torch

In [3]:
# In order to get the best performance, deep learning frameworks are memory greedy by default.
# To avoid out of memory errors, the following code reduces the amount of memory pre-allocated
# by JAX, TensorFlow and PyTorch.

# Allocate 25% of the memory to each framework
FRACTION=0.25 

# JAX
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']=str(FRACTION)

# TensorFlow
tf.config.set_logical_device_configuration(tf.config.list_physical_devices('GPU')[0], 
   [tf.config.LogicalDeviceConfiguration(memory_limit=torch.cuda.get_device_properties(0).total_memory * FRACTION/1024/1024)])

# Quiet warning from MxNet
os.environ["MXNET_CUDNN_LIB_CHECKING"]='0'


<h2><a id='index'></a>Index<a href='#top' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

<table style='table-layout: fixed; width=100'>
    <colgroup>
        <col style="width:10%"><col style="width:10%"><col style="width:10%"><col style="width:10%"><col style="width:10%">
        <col style="width:10%"><col style="width:10%"><col style="width:10%"><col style="width:10%"><col style="width:10%">
      </colgroup>  
    <thead><tr><th></th><th>Pandas</th><th>Numpy</th><th>cuDF</th><th>cuPY</th><th>JAX</th><th>Numba</th><th>TensorFlow</th><th>PyTorch</th><th>MXNet</th></tr></thead><tbody>
     <tr><th align='left'>Pandas</th>      <td align='center'>n/a</td>                                   <td align='center'><a href='#numpy-pandas'>code</a></td>     <td align='center'><a href='#cudf-pandas'>code</a></td>     <td align='center'><a href='#cupy-pandas'>code</a></td>     <td align='center'><a href='#jax-pandas'>code</a></td>     <td align='center'><a href='#numba-pandas'>code</a></td>     <td align='center'><a href='#tensorflow-pandas'>code</a></td>  <td align='center'><a href='#pytorch-pandas'>code</a></td>     <td align='center'><a href='#mxnet-pandas'>code</a></td></tr>
     <tr><th align='left'>Numpy</th>       <td align='center'><a href='#pandas-numpy'>code</a></td>      <td align='center'>n/a</td>                                  <td align='center'><a href='#cudf-numpy'>code</a></td>      <td align='center'><a href='#cupy-numpy'>code</a></td>      <td align='center'><a href='#jax-numpy'>code</a></td>      <td align='center'><a href='#numba-numpy'>code</a></td>      <td align='center'><a href='#tensorflow-numpy'>code</a></td>   <td align='center'><a href='#pytorch-numpy'>code</a></td>      <td align='center'><a href='#mxnet-numpy'>code</a></td></tr>
     <tr><th align='left'>cuDF</th>        <td align='center'><a href='#pandas-cudf'>code</a></td>       <td align='center'><a href='#numpy-cudf'>code</a></td>       <td align='center'>n/a</td>                                 <td align='center'><a href='#cupy-cudf'>code</a></td>       <td align='center'><a href='#jax-cudf'>code</a></td>       <td align='center'><a href='#numba-cudf'>code</a></td>       <td align='center'><a href='#tensorflow-cudf'>code</a></td>    <td align='center'><a href='#pytorch-cudf'>code</a></td>       <td align='center'><a href='#mxnet-cudf'>code</a></td></tr>
     <tr><th align='left'>cuPY</th>        <td align='center'><a href='#pandas-cupy'>code</a></td>       <td align='center'><a href='#numpy-cupy'>code</a></td>       <td align='center'><a href='#cudf-cupy'>code</a></td>       <td align='center'>n/a</td>                                 <td align='center'><a href='#jax-cupy'>code</a></td>       <td align='center'><a href='#numba-cupy'>code</a></td>       <td align='center'><a href='#tensorflow-cupy'>code</a></td>    <td align='center'><a href='#pytorch-cupy'>code</a></td>       <td align='center'><a href='#mxnet-cupy'>code</a></td></tr>
     <tr><th align='left'>JAX</th>         <td align='center'><a href='#pandas-jax'>code</a></td>        <td align='center'><a href='#numpy-jax'>code</a></td>        <td align='center'><a href='#cudf-jax'>code</a></td>        <td align='center'><a href='#cupy-jax'>code</a></td>        <td align='center'>n/a</td>                                <td align='center'><a href='#numba-jax'>code</a></td>        <td align='center'><a href='#tensorflow-jax'>code</a></td>     <td align='center'><a href='#pytorch-jax'>code</a></td>        <td align='center'><a href='#mxnet-jax'>code</a></td></tr>
     <tr><th align='left'>Numba</th>       <td align='center'><a href='#pandas-numba'>code</a></td>      <td align='center'><a href='#numpy-numba'>code</a></td>      <td align='center'><a href='#cudf-numba'>code</a></td>      <td align='center'><a href='#cupy-numba'>code</a></td>      <td align='center'><a href='#jax-numba'>code</a></td>      <td align='center'>n/a</td>                                  <td align='center'><a href='#tensorflow-numba'>code</a></td>   <td align='center'><a href='#pytorch-numba'>code</a></td>      <td align='center'><a href='#mxnet-numba'>code</a></td></tr>
     <tr><th align='left'>TensorFlow</th>  <td align='center'><a href='#pandas-tensorflow'>code</a></td> <td align='center'><a href='#numpy-tensorflow'>code</a></td> <td align='center'><a href='#cudf-tensorflow'>code</a></td> <td align='center'><a href='#cupy-tensorflow'>code</a></td> <td align='center'><a href='#jax-tensorflow'>code</a></td> <td align='center'><a href='#numba-tensorflow'>code</a></td> <td align='center'>n/a</td>                                    <td align='center'><a href='#pytorch-tensorflow'>code</a></td> <td align='center'><a href='#mxnet-tensorflow'>code</a></td></tr>
     <tr><th align='left'>PyTorch</th>     <td align='center'><a href='#pandas-pytorch'>code</a></td>    <td align='center'><a href='#numpy-pytorch'>code</a></td>    <td align='center'><a href='#cudf-pytorch'>code</a></td>    <td align='center'><a href='#cupy-pytorch'>code</a></td>    <td align='center'><a href='#jax-pytorch'>code</a></td>    <td align='center'><a href='#numba-pytorch'>code</a></td>    <td align='center'><a href='#tensorflow-pytorch'>code</a></td> <td align='center'>n/a</td>                                    <td align='center'><a href='#mxnet-pytorch'>code</a></td></tr>
     <tr><th align='left'>MXNet</th>       <td align='center'><a href='#pandas-mxnet'>code</a></td>      <td align='center'><a href='#numpy-mxnet'>code</a></td>      <td align='center'><a href='#cudf-mxnet'>code</a></td>      <td align='center'><a href='#cupy-mxnet'>code</a></td>      <td align='center'><a href='#jax-mxnet'>code</a></td>      <td align='center'><a href='#numba-mxnet'>code</a></td>      <td align='center'><a href='#tensorflow-mxnet'>code</a></td>   <td align='center'><a href='#pytorch-mxnet'>code</a></td>      <td align='center'>n/a</td></tr>
    </tbody>
</table>

<h2><a id='pandas-numpy'></a>From Pandas to Numpy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [4]:
# Option 1: Pandas DataFrame to a Numpy ndarray
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = src.to_numpy()

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 3.]
 [2. 4.]]


In [5]:
# Option 2: Convert a Pandas DataFrame to a Numpy ndarray
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = src.values # "to_numpy()" is preferred to "values".

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 3.]
 [2. 4.]]


In [6]:
# Option 3: Convert a Pandas DataFrame to a Numpy recarray
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]}, index=['a', 'b'])
dst = src.to_records()

print(type(dst), "\n", dst)

<class 'numpy.recarray'> 
 [('a', 1., 3.) ('b', 2., 4.)]


<h2><a id='pandas-cudf'></a>From Pandas to cuDF<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [7]:
# Option 1: Convert a Pandas DataFrame to a cuDF DataFrame
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = cudf.DataFrame(src)

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      x    y
0  1.0  3.0
1  2.0  4.0


In [8]:
# Option 2: Convert a Pandas DataFrame to a cuDF DataFrame
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = cudf.from_pandas(src)

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      x    y
0  1.0  3.0
1  2.0  4.0


<h2><a id='pandas-cupy'></a>From Pandas to CuPy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [9]:
# Option 1: Pandas DataFrame to a CuPy ndarray
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = cp.asarray(src)

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 3.]
 [2. 4.]]


In [10]:
# Option 2: Pandas DataFrame to a CuPy ndarray
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = cp.array(src)

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 3.]
 [2. 4.]]


<h2><a id='pandas-jax'></a>From Pandas to Jax<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

Jax does not natively support Pandas DataFrames. Nevertheless, it supports Numpy ndarrays, which can be generated from Pandas DataFrames.

See: Pandas → [Numpy](#pandas-numpy) → [Jax](#numpy-jax).

<h2><a id='pandas-numba'></a>From Pandas to Numba<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

Numba does not natively support Pandas DataFrames. Nevertheless, a Pandas DataFrame can be converted to other Numba-supported formats.

- Pandas → [cuDF](#pandas-cudf) → [Numba](#cudf-numba)
- Pandas → [CuPy](#pandas-cupy) → [Numba](#cupy-numba)
- Pandas → [JAX](#pandas-jax) → [Numba](#jax-numba)
- Pandas → [Numpy](#pandas-numpy) → [Numba](#numpy-numba)

<h2><a id='pandas-tensorflow'></a>From Pandas to TensorFlow<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [11]:
# Convert a Pandas DataFrame to a TensorFlow EagerTensor
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = tf.convert_to_tensor(src.values)


print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 3.]
 [2. 4.]], shape=(2, 2), dtype=float64)


<h2><a id='pandas-pytorch'></a>From Pandas to PyTorch<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

PyTorch does not natively support Pandas DataFrames. Nevertheless, it supports Numpy ndarrays, which can be generated from Pandas DataFrames.

See: Pandas → [Numpy](#pandas-numpy) → [PyTorch](#numpy-pytorch).

<h2><a id='pandas-mxnet'></a>From Pandas to MxNet<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [12]:
# Convert a Pandas DataFrame to an MxNet NDArray
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = mx.nd.array(src, ctx=mx.gpu())

print(type(dst), "\n", dst)

<class 'mxnet.ndarray.ndarray.NDArray'> 
 
[[1. 3.]
 [2. 4.]]
<NDArray 2x2 @gpu(0)>


<h2><a id='numpy-pandas'></a>From Numpy to Pandas<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [13]:
# Convert a Numpy ndarray to a Pandas DataFrame
src = np.array([[1., 2.], [3., 4.]])
dst = pd.DataFrame(src)

print(type(dst), "\n", dst)

<class 'pandas.core.frame.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='numpy-cudf'></a>From Numpy to cuDF<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [14]:
# Option 1: Numpy ndarray to a Pandas DataFrame
src = np.array([[1., 2.], [3., 4.]])
dst = cudf.DataFrame(src)

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


In [15]:
# Option 2: Numpy recarray to a Pandas DataFrame
src = np.rec.array([(1., 2.), (3., 4.)], names=['a', 'b'])
dst = cudf.DataFrame.from_records(src)

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      a    b
0  1.0  2.0
1  3.0  4.0


<h2><a id='numpy-cupy'></a>From Numpy to CuPy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [16]:
# Option 1: Numpy ndarray to a CuPy ndarray
src = np.array([[1., 2.], [3., 4.]])
dst = cp.asarray(src)

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [17]:
# Option 2: Numpy ndarray to a CuPy ndarray
src = np.array([[1., 2.], [3., 4.]])
dst = cp.array(src)

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='numpy-jax'></a>From Numpy to JAX<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [18]:
# Option 1: Numpy ndarray to a JAX DeviceArray
src = np.array([[1., 2.], [3., 4.]])
dst = jnp.array(src)

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


In [19]:
# Option 2: Numpy ndarray to a JAX DeviceArray
src = np.array([[1., 2.], [3., 4.]])
dst = jnp.asarray(src)

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


In [20]:
# Option 3: Numpy ndarray to a JAX DeviceArray
src = np.array([[1., 2.], [3., 4.]])
dst = jax.device_put(src)

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='numpy-numba'></a>From Numpy to Numba<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

Numba natively supports Numpy `ndarray`s. Alternatively, a Numba ``DeviceNDArray` can be created from a Numpy ndarray. 

In [21]:
# Convert a Numpy ndarray to a Numba DeviceNDArray
src = np.array([[1., 2.], [3., 4.]])
dst = nb.cuda.to_device(src)

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='numpy-tensorflow'></a>From Numpy to TensorFlow<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [22]:
# Convert a Numpy ndarray to a TensorFlow EagerTensor
src = np.array([[1., 2.], [3., 4.]])
dst = tf.convert_to_tensor(src)

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float64)


<h2><a id='numpy-pytorch'></a>From Numpy to PyTorch<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [23]:
# Convert a Numpy ndarray to a PyTorch Tensor
src = np.array([[1., 2.], [3., 4.]])
dst = torch.tensor(src)

print(type(dst), "\n", dst)

<class 'torch.Tensor'> 
 tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


<h2><a id='numpy-mxnet'></a>From Numpy to MxNet<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [24]:
# Convert a Numpy ndarray to a MxNet NDArray
src = np.array([[1., 2.], [3., 4.]])
dst = mx.nd.array(src, ctx=mx.gpu())

print(type(dst), "\n", dst)

<class 'mxnet.ndarray.ndarray.NDArray'> 
 
[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @gpu(0)>


<h2><a id='cudf-pandas'></a>From cuDF to Pandas<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [25]:
# Convert a cuDF DataFrame to a Pandas DataFrame
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = src.to_pandas()

print(type(dst), "\n", dst)

<class 'pandas.core.frame.DataFrame'> 
      x    y
0  1.0  3.0
1  2.0  4.0


<h2><a id='cudf-numpy'></a>From cuDF to Numpy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [26]:
# Option 1: Convert a cuDF DataFrame to a Numpy ndarray
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = src.to_numpy()

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 3.]
 [2. 4.]]


In [27]:
# Option 2: Convert a cuDF DataFrame to a Numpy recarray
src = pd.DataFrame({'x': [1., 2.], 'y': [3., 4.]}, index=['a', 'b'])
dst = src.to_records()

print(type(dst), "\n", dst)

<class 'numpy.recarray'> 
 [('a', 1., 3.) ('b', 2., 4.)]


<h2><a id='cudf-cupy'></a>From cuDF to CuPy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [28]:
# Option 1: Convert a cuDF DataFrame to a CuPy ndarray
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = src.to_cupy()

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 3.]
 [2. 4.]]


In [29]:
# Option 2: Convert a cuDF DataFrame to a CuPy ndarray
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = cp.from_dlpack(src.to_dlpack())

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 3.]
 [2. 4.]]


<h2><a id='cudf-jax'></a>From cuDF to JAX<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [30]:
# Convert a cuDF DataFrame to a JAX DeviceArray
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = jax.dlpack.from_dlpack(src.to_dlpack())

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 3.]
 [2. 4.]]


<h2><a id='cudf-numba'></a>From cuDF to Numba<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [31]:
# Convert a cuDF DataFrame to a Numba DeviceNDArray
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = nb.cuda.as_cuda_array(src.to_cupy())

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 3.]
 [2. 4.]]


<h2><a id='cudf-tensorflow'></a>From cuDF to TensorFlow<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

TensorFlow does not natively column-major unit-stride data. Therefore, we need to perform a cuDF to CuPy conversion (zero-copy operation), then transpose and make a copy of the data to finally convert from CuPy to DLPack (zero-copy operation).

In [32]:
# Option 1: Convert a cuDF DataFrame to a TensorFlow EagerTensor
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = tf.experimental.dlpack.from_dlpack(src.to_cupy().T.copy().toDlpack())

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float64)


In [33]:
# Option 2: Convert a cuDF DataFrame to a TensorFlow EagerTensor
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = tf.experimental.dlpack.from_dlpack(cp.from_dlpack(src.to_dlpack()).T.toDlpack())

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float64)


<h2><a id='cudf-pytorch'></a>From cuDF to PyTorch<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [34]:
# Convert a cuDF DataFrame to a PyTorch Tensor
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = torch.from_dlpack(src.to_dlpack())

print(type(dst), "\n", dst)

<class 'torch.Tensor'> 
 tensor([[1., 3.],
        [2., 4.]], device='cuda:0', dtype=torch.float64)


<h2><a id='cudf-mxnet'></a>From cuDF to MxNet<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [35]:
# Convert a cuDF DataFrame to an MxNet NDArray
src = cudf.DataFrame({'x': [1., 2.], 'y': [3., 4.]})
dst = mx.nd.array(src.to_numpy(), ctx=mx.gpu())

print(type(dst), "\n", dst)

<class 'mxnet.ndarray.ndarray.NDArray'> 
 
[[1. 3.]
 [2. 4.]]
<NDArray 2x2 @gpu(0)>


<h2><a id='cupy-pandas'></a>From CuPy to Pandas<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [36]:
# Option 1: Convert a CuPy ndarray to a Pandas DataFrame 
src = cp.array([[1., 2.], [3., 4.]])
dst = pd.DataFrame(src.get())

print(type(dst), "\n", dst)

<class 'pandas.core.frame.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


In [37]:
# Option 2: Convert a CuPy ndarray to a Pandas DataFrame 
src = cp.array([[1., 2.], [3., 4.]])
dst = pd.DataFrame(cp.ndarray.get(src))

print(type(dst), "\n", dst)

<class 'pandas.core.frame.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


In [38]:
# Option 3: Convert a CuPy ndarray to a Pandas DataFrame 
src = cp.array([[1., 2.], [3., 4.]])
dst = pd.DataFrame(cp.asnumpy(src))

print(type(dst), "\n", dst)

<class 'pandas.core.frame.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='cupy-numpy'></a>From CuPy to Numpy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [39]:
# Option 1: Convert a CuPy ndarray to a Numpy ndarray 
src = cp.array([[1., 2.], [3., 4.]])
dst = cp.asnumpy(src)

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [40]:
# Option 2: Convert a CuPy ndarray to a Numpy ndarray 
src = cp.array([[1., 2.], [3., 4.]])
dst = src.get()

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [41]:
# Option 3: Convert a CuPy ndarray to a Numpy ndarray 
src = cp.array([[1., 2.], [3., 4.]])
dst = cp.ndarray.get(src)

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='cupy-cudf'></a>From CuPy to cuDF<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [42]:
# Option 1: Convert a CuPy ndarray to a cuDF DataFrame 
src = cp.array([[1., 2.], [3., 4.]])
dst = cudf.DataFrame(src)

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


In [43]:
# Option 2: Convert a CuPy ndarray to a cuDF DataFrame 
src = cp.array([[1., 2.], [3., 4.]], order='F')
dst = cudf.from_dlpack(src.toDlpack())

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='cupy-jax'></a>From CuPy to JAX<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [44]:
# Convert a CuPy ndarray to a JAX DeviceArray
src = cp.array([[1., 2.], [3., 4.]])
dst = jax.dlpack.from_dlpack(src.toDlpack())

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='cupy-numba'></a>From CuPy to Numba<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [45]:
# Option 1: Convert a CuPy ndarray to a Numba DeviceNDArray
src = cp.array([[1., 2.], [3., 4.]])
dst = nb.cuda.as_cuda_array(src)

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 2.]
 [3. 4.]]


In [46]:
# Option 2: Convert a CuPy ndarray to a Numba DeviceNDArray
src = cp.array([[1., 2.], [3., 4.]])
dst = nb.cuda.to_device(src)

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='cupy-tensorflow'></a>From CuPy to TensorFlow<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [47]:
# Convert a CuPy ndarray to a TensorFlow EagerTensor
src = cp.array([[1., 2.], [3., 4.]])
dst = tf.experimental.dlpack.from_dlpack(src.toDlpack())

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float64)


<h2><a id='cupy-pytorch'></a>From CuPy to PyTorch<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [48]:
# Convert a CuPy ndarray to a PyTorch Tensor
src = cp.array([[1., 2.], [3., 4.]])
dst = torch.as_tensor(src, device='cuda')

print(type(dst), "\n", dst)

<class 'torch.Tensor'> 
 tensor([[1., 2.],
        [3., 4.]], device='cuda:0', dtype=torch.float64)


In [49]:
# Convert a CuPy ndarray to a PyTorch Tensor
src = cp.array([[1., 2.], [3., 4.]])
dst = torch.from_dlpack(src.toDlpack())

print(type(dst), "\n", dst)

<class 'torch.Tensor'> 
 tensor([[1., 2.],
        [3., 4.]], device='cuda:0', dtype=torch.float64)


<h2><a id='cupy-mxnet'></a>From CuPy to MxNet<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [50]:
# Convert a CuPy ndarray to a MxNet NDArray
src = cp.array([[1., 2.], [3., 4.]])
dst = mx.nd.from_dlpack(src.toDlpack())

print(type(dst), "\n", dst)

<class 'mxnet.ndarray.ndarray.NDArray'> 
 
[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @gpu(0)>


<h2><a id='jax-pandas'></a>From JAX to Pandas<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [51]:
# Convert a JAX DeviceArray to a Pandas DataFrame
src = jnp.array([[1., 2.], [3., 4.]])
dst = pd.DataFrame(src)

print(type(dst), "\n", dst)

<class 'pandas.core.frame.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='jax-numpy'></a>From JAX to Numpy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [52]:
# Option 1: Convert a JAX DeviceArray to a Numpy ndarray
src = jnp.array([[1., 2.], [3., 4.]])
dst = np.asarray(src)

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [53]:
# Option 2: Convert a JAX DeviceArray to a Numpy ndarray
src = jnp.array([[1., 2.], [3., 4.]])
dst = np.array(src)

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='jax-cudf'></a>From JAX to cuDF<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [54]:
# Convert a JAX DeviceArray to a cuDF DataFrame
src = jnp.array([[1., 2.], [3., 4.]])
dst = cudf.DataFrame(cp.from_dlpack(jax.dlpack.to_dlpack(src)))

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='jax-cupy'></a>From JAX to CuPy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [55]:
# Convert a JAX DeviceArray to a CuPY ndarray
src = jnp.array([[1., 2.], [3., 4.]])
dst = cp.from_dlpack(jax.dlpack.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='jax-numba'></a>From JAX to Numba<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [56]:
# Option 1: Convert a JAX DeviceArray to a Numba DeviceNDArray
src = jnp.array([[1., 2.], [3., 4.]])
dst = nb.cuda.as_cuda_array(src)

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 2.]
 [3. 4.]]


In [57]:
# Option 2: Convert a JAX DeviceArray to a Numba DeviceNDArray
src = jnp.array([[1., 2.], [3., 4.]])
dst = nb.cuda.to_device(src)

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='jax-tensorflow'></a>From JAX to TensorFlow<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [58]:
# Convert a JAX DeviceArray to a TensorFlow EagerTensor
src = jnp.array([[1., 2.], [3., 4.]])
dst = tf.experimental.dlpack.from_dlpack(jax.dlpack.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


<h2><a id='jax-pytorch'></a>From JAX to PyTorch<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [59]:
# Convert a JAX DeviceArray to a PyTorch Tensor
src = jnp.array([[1., 2.], [3., 4.]])
dst = torch.from_dlpack(jax.dlpack.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'torch.Tensor'> 
 tensor([[1., 2.],
        [3., 4.]], device='cuda:0')


<h2><a id='jax-mxnet'></a>From JAX to MxNet<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [60]:
# Convert a JAX DeviceArray to a MxNet NDArray
src = jnp.array([[1., 2.], [3., 4.]])
dst = mx.nd.from_dlpack(jax.dlpack.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'mxnet.ndarray.ndarray.NDArray'> 
 
[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @gpu(0)>


<h2><a id='numba-pandas'></a>From Numba to Pandas<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

Pandas does not natively support Numba DeviceNDArrays. Nevertheless, it supports Numpy ndarrays, which can be generated from Numba DeviceNDArrays:

- Numba → [Numpy](#numba-numpy) → [Pandas](#numpy-pandas)

<h2><a id='numba-numpy'></a>From Numba to Numpy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [61]:
# Convert a GPU-based Numba DeviceNDArray to a Numpy ndarray
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = src.copy_to_host()

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='numba-cudf'></a>From Numba to cuDF<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [62]:
# Convert a GPU-based Numba DeviceNDArray to a cuDF DataFrame
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = cudf.DataFrame(src)

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


In [63]:
# Convert a GPU-based Numba DeviceNDArray to a cuDF DataFrame
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = cudf.DataFrame(src)

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='numba-cupy'></a>From Numba to CuPy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [64]:
# Option 1: Convert a GPU-based Numba DeviceNDArray to a CuPy ndarray
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = cp.asarray(src)

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [65]:
# Option 2: Convert a GPU-based Numba DeviceNDArray to a CuPy ndarray
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = cp.array(src)

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='numba-jax'></a>From Numba to JAX<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [66]:
# Option 1: Convert a GPU-based Numba DeviceNDArray to a JAX DeviceArray
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = jnp.asarray(src)

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


In [67]:
# Option 2: Convert a GPU-based Numba DeviceNDArray to a JAX DeviceArray
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = jnp.array(src)

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='numba-tensorflow'></a>From Numba to TensorFlow<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [68]:
# Convert a GPU-based Numba DeviceNDArray to a TensorFlow EagerTensor
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = tf.convert_to_tensor(src)

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float64)


<h2><a id='numba-pytorch'></a>From Numba to PyTorch<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [69]:
# Convert a GPU-based Numba DeviceNDArray to a PyTorch Tensor
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = torch.tensor(src)

print(type(dst), "\n", dst)

<class 'torch.Tensor'> 
 tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


<h2><a id='numba-mxnet'></a>From Numba to MxNet<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [70]:
# Convert a GPU-based Numba DeviceNDArray to a MxNet NDArray
src = nb.cuda.to_device([[1., 2.], [3., 4.]])
dst = mx.nd.array(src, ctx=mx.gpu())

print(type(dst), "\n", dst)

<class 'mxnet.ndarray.ndarray.NDArray'> 
 
[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @gpu(0)>


<h2><a id='tensorflow-pandas'></a>From TensorFlow to Pandas<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

Pandas does not natively support TensorFlow EagerTensors. Nevertheless, it supports Numpy ndarrays, which can be generated from TensorFlow EagerTensors:

- TensorFlow → [Numpy](#tensorflow-numpy) → [Pandas](#numpy-pandas)

<h2><a id='tensorflow-numpy'></a>From TensorFlow to Numpy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [71]:
# Option 1: Convert a TensorFlow EagerTensor to a Numpy ndarray
src = tf.convert_to_tensor([[1., 2.], [3., 4.]])
dst = np.asarray(src)

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [72]:
# Option 2: Convert a TensorFlow EagerTensor to a Numpy ndarray
src = tf.convert_to_tensor([[1., 2.], [3., 4.]])
dst = np.array(src)

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [73]:
# Option 3: Convert a TensorFlow EagerTensor to a Numpy ndarray
src = tf.convert_to_tensor([[1., 2.], [3., 4.]])
dst = src.numpy()

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='tensorflow-cudf'></a>From TensorFlow to cuDF<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [74]:
# Convert a TensorFlow EagerTensor to a cuDF DataFrame
src = tf.convert_to_tensor([[1., 2.], [3., 4.]])
dst = cudf.DataFrame(cp.from_dlpack(tf.experimental.dlpack.to_dlpack(src)))

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='tensorflow-cupy'></a>From TensorFlow to CuPy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [75]:
# Convert a TensorFlow EagerTensor to a CuPy ndarray
src = tf.math.add(tf.zeros([2, 2]), [[1., 2.], [3., 4.]])
dst = cp.from_dlpack(tf.experimental.dlpack.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='tensorflow-jax'></a>From TensorFlow to JAX<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [76]:
# Option 1: Convert a GPU-based TensorFlow EagerTensor to a GPU-based JAX DeviceArray
src = tf.math.add(tf.zeros([2, 2]), [[1., 2.], [3., 4.]])
dst = jax.dlpack.from_dlpack(tf.experimental.dlpack.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


In [77]:
# Option 2: Convert a CPU or GPU-based TensorFlow EagerTensor to a CPU-based JAX DeviceArray
src = tf.convert_to_tensor([[1., 2.], [3., 4.]])
dst = jnp.asarray(src)

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


In [78]:
# Option 3: Convert a CPU or GPU-based TensorFlow EagerTensor to a CPU-based JAX DeviceArray
src = tf.convert_to_tensor([[1., 2.], [3., 4.]])
dst = jnp.array(src)

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='tensorflow-numba'></a>From TensorFlow to Numba<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [79]:
# Convert a TensorFlow EagerTensor to a Numba DeviceNDArray
src = tf.convert_to_tensor([[1., 2.], [3., 4.]])
dst = nb.cuda.to_device(src)

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='tensorflow-pytorch'></a>From TensorFlow to PyTorch<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [80]:
# Convert a TensorFlow EagerTensor to a PyTorch Tensor
src = tf.math.add(tf.zeros([2, 2]), [[1., 2.], [3., 4.]])
dst = torch.from_dlpack(tf.experimental.dlpack.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'torch.Tensor'> 
 tensor([[1., 2.],
        [3., 4.]], device='cuda:0')


<h2><a id='tensorflow-mxnet'></a>From TensorFlow to MxNet<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [81]:
# Convert a TensorFlow EagerTensor to a Numba DeviceNDArray
src = tf.math.add(tf.zeros([2, 2]), [[1., 2.], [3., 4.]])
dst = mx.nd.from_dlpack(tf.experimental.dlpack.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'mxnet.ndarray.ndarray.NDArray'> 
 
[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @gpu(0)>


<h2><a id='pytorch-pandas'></a>From PyTorch to Pandas<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [82]:
# Convert a PyTorch Tensor to a Pandas DataFrame
src = torch.tensor([[1., 2.], [3., 4.]])
dst = pd.DataFrame(src).astype("float")

print(type(dst), "\n", dst)

<class 'pandas.core.frame.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='pytorch-numpy'></a>From PyTorch to Numpy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [83]:
# Option 1: Convert a GPU-based PyTorch Tensor to a Numpy ndarray
src = torch.cuda.FloatTensor([[1., 2.], [3., 4.]])
dst = src.cpu().numpy()

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [84]:
# Option 2: Convert a CPU-based PyTorch Tensor to a Numpy ndarray
src = torch.tensor([[1., 2.], [3., 4.]])
dst = src.numpy()

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='pytorch-cudf'></a>From PyTorch to cuDF<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [85]:
# Convert a PyTorch Tensor to a cuDF DataFrame
src = torch.cuda.FloatTensor([[1., 2.], [3., 4.]])
dst = cudf.DataFrame(src)

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='pytorch-cupy'></a>From PyTorch to CuPy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [86]:
# Option 1: Convert a CPU or GPU-based PyTorch Tensor to a CuPy ndarray
src = torch.tensor([[1., 2.], [3., 4.]], dtype=torch.float)
dst = cp.asarray(src)

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [87]:
# Option 2: Convert a CPU or GPU-based PyTorch Tensor to a CuPy ndarray
src = torch.cuda.FloatTensor([[1., 2.], [3., 4.]])
dst = cp.array(src)

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [88]:
# Option 3: Convert a GPU-based PyTorch Tensor to a CuPy ndarray
src = torch.cuda.FloatTensor([[1., 2.], [3., 4.]])
dst = cp.from_dlpack(torch.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='pytorch-jax'></a>From PyTorch to JAX<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [89]:
# Option 1: Convert a CPU-based PyTorch Tensor to a JAX DeviceArray
src = torch.tensor([[1., 2.], [3., 4.]], dtype=torch.float)
dst = jnp.asarray(src)

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


In [90]:
# Option 2: Convert a CPU-based PyTorch Tensor to a JAX DeviceArray
src = torch.tensor([[1., 2.], [3., 4.]], dtype=torch.float)
dst = jnp.array(src)

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


In [91]:
# Option 3: Convert a GPU-based PyTorch Tensor to a JAX DeviceArray
src = torch.cuda.FloatTensor([[1., 2.], [3., 4.]])
dst = jax.dlpack.from_dlpack(torch.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='pytorch-numba'></a>From PyTorch to Numba<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [92]:
# Option 1: Convert a GPU-based PyTorch Tensor to a Numba DeviceNDArray
src = torch.cuda.FloatTensor([[1., 2.], [3., 4.]])
dst = nb.cuda.as_cuda_array(src)

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 2.]
 [3. 4.]]


In [93]:
# Option 2: Convert a CPU or GPU-based PyTorch Tensor to a Numba DeviceNDArray
src = torch.tensor([[1., 2.], [3., 4.]], dtype=torch.float)
dst = nb.cuda.to_device(src)

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='pytorch-tensorflow'></a>From PyTorch to TensorFlow<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [94]:
# Option 1: Convert a CPU or GPU-based PyTorch Tensor to a TensorFlow EagerTensor
src = torch.cuda.FloatTensor([[1., 2.], [3., 4.]])
dst = tf.experimental.dlpack.from_dlpack(torch.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


In [95]:
# Option 2: Convert a CPU-based PyTorch Tensor to a TensorFlow EagerTensor
src = torch.tensor([[1., 2.], [3., 4.]], dtype=torch.float)
dst = tf.convert_to_tensor(src)

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


<h2><a id='pytorch-mxnet'></a>From PyTorch to MxNet<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [96]:
# Option 1: Convert a CPU-based PyTorch Tensor to a TensorFlow EagerTensor
src = torch.tensor([[1., 2.], [3., 4.]], dtype=torch.float)
dst = mx.nd.array(src, ctx=mx.gpu())

print(type(dst), "\n", dst)

<class 'mxnet.ndarray.ndarray.NDArray'> 
 
[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @gpu(0)>


In [97]:
# Option 2: Convert a CPU or GPU-based PyTorch Tensor to a TensorFlow EagerTensor
src = torch.cuda.FloatTensor([[1., 2.], [3., 4.]])
dst = mx.nd.from_dlpack(torch.to_dlpack(src))

print(type(dst), "\n", dst)

<class 'mxnet.ndarray.ndarray.NDArray'> 
 
[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @gpu(0)>


<h2><a id='mxnet-pandas'></a>From MxNet to Pandas<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

Pandas does not natively support MxNet NDArrays. Nevertheless, it supports Numpy ndarrays, which can be generated from MxNet NDArrays.

See: MxNet → [Numpy](#mxnet-numpy) → [Pandas](#numpy-pandas).

<h2><a id='mxnet-numpy'></a>From MxNet to Numpy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [98]:
#Convert a CPU or GPU-based MxNet NDArray to a Numpy ndarray
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = src.asnumpy()

print(type(dst), "\n", dst)

<class 'numpy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='mxnet-cudf'></a>From MxNet to cuDF<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [99]:
# Option 1: Convert a CPU or GPU-based MxNet NDArray to a cuDF DataFrame
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = cudf.DataFrame(cp.from_dlpack(src.to_dlpack_for_write()))

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


In [100]:
# Option 2: Convert a CPU or GPU-based MxNet NDArray to a cuDF DataFrame
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = cudf.DataFrame(cp.from_dlpack(src.to_dlpack_for_read()))

print(type(dst), "\n", dst)

<class 'cudf.core.dataframe.DataFrame'> 
      0    1
0  1.0  2.0
1  3.0  4.0


<h2><a id='mxnet-cupy'></a>From MxNet to CuPy<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

Cupy does not natively support CPU-based MxNet NDArrays. Nevertheless, it supports Numpy ndarrays, which can be generated from MxNet NDArrays.

See: MxNet → [Numpy](#mxnet-numpy) → [CuPy](#numpy-cupy).

In [101]:
# Option 1: Convert a GPU-based MxNet NDArray to a CuPy ndarray
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = cp.from_dlpack(src.to_dlpack_for_write())

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


In [102]:
# Option 2: Convert a GPU-based MxNet NDArray to a CuPy ndarray
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = cp.from_dlpack(src.to_dlpack_for_read())

print(type(dst), "\n", dst)

<class 'cupy.ndarray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='mxnet-jax'></a>From MxNet to JAX<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

JAX does not natively support CPU-based MxNet NDArrays. Nevertheless, it supports Numpy ndarrays, which can be generated from MxNet NDArrays.

See: MxNet → [Numpy](#mxnet-numpy) → [JAX](#numpy-jax).

In [103]:
# Option 1: Convert a GPU-based MxNet NDArray to a CuPy ndarray
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = jax.dlpack.from_dlpack(src.to_dlpack_for_write())

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


In [104]:
# Option 2: Convert a GPU-based MxNet NDArray to a CuPy ndarray
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = jax.dlpack.from_dlpack(src.to_dlpack_for_read())

print(type(dst), "\n", dst)

<class 'jaxlib.xla_extension.DeviceArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='mxnet-numba'></a>From MxNet to Numba<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [105]:
# Convert a CPU or GPU-based MxNet NDArray to a Numba DeviceNDArray
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = nb.cuda.to_device(cp.from_dlpack(src.to_dlpack_for_write()))

print(type(dst), "\n", dst.copy_to_host())

<class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'> 
 [[1. 2.]
 [3. 4.]]


<h2><a id='mxnet-tensorflow'></a>From MxNet to TensorFlow<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [106]:
# Option 1 - Convert a CPU or GPU-based MxNet NDArray to a TensorFlow EagerTensor
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = tf.experimental.dlpack.from_dlpack(src.to_dlpack_for_write())

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float16)


In [107]:
# Option 2 - Convert a CPU or GPU-based MxNet NDArray to a TensorFlow EagerTensor
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = tf.experimental.dlpack.from_dlpack(src.to_dlpack_for_read())

print(type(dst), "\n", dst)

<class 'tensorflow.python.framework.ops.EagerTensor'> 
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float16)


<h2><a id='mxnet-pytorch'></a>From MxNet to PyTorch<a href='#index' style='position:absolute;right:0;top:55%;font-size:small'>↑↑↑</a></h2>

In [108]:
# Option 1 - Convert a CPU or GPU-based MxNet NDArray to a PyTorch Tensor
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = torch.from_dlpack(src.to_dlpack_for_write())

print(type(dst), "\n", dst)

<class 'torch.Tensor'> 
 tensor([[1., 2.],
        [3., 4.]], device='cuda:0', dtype=torch.float16)


In [109]:
# Option 2 - Convert a CPU or GPU-based MxNet NDArray to a PyTorch Tensor
src = mx.nd.array([[1., 2.], [3., 4.]], dtype='float16', ctx=mx.gpu())
dst = torch.from_dlpack(src.to_dlpack_for_read())

print(type(dst), "\n", dst)

<class 'torch.Tensor'> 
 tensor([[1., 2.],
        [3., 4.]], device='cuda:0', dtype=torch.float16)
